In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `bootpython && heroku run --app guarded-everglades-89687 make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run --app guarded-everglades-89687 make importdb && make pipeline`
# (6) Remember to delete contents of ~/Documents/links.txt
links = pd.read_csv('data/export.csv')
links

/var/folders/7f/6ty6wfcd1hs3d1wz9b1c4gsw0000gn/T/ipykernel_2999/1024356683.py:12: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108364,172995,https://split-ticket.org/2024/01/12/the-potent...,The Potential Political Impact of Shifting Sta...,\N,\N,2024-01-13 07:52:15.630927+00,2024-01-13 07:52:15.630927+00,NaN,\N,SplitTicket,24.0,0.0,\N
108365,172996,https://www.metaculus.com/news/2024/01/12/news...,newstestjan,\N,\N,2024-01-13 07:52:15.631894+00,2024-01-13 07:52:15.631894+00,NaN,\N,Metaculus,13.0,0.0,\N
108366,172997,https://www.slowboring.com/p/mailbags-and-movi...,Mailbags (and movies) need writers,\N,\N,2024-01-13 07:52:15.632853+00,2024-01-13 07:52:15.632853+00,NaN,\N,Yglesias,98.0,0.0,\N
108367,172998,https://1a3orn.com/sub/machine-learning-bans.html,Many AI Safety Orgs Have Tried to Criminalize ...,\N,\N,2024-01-13 07:52:15.634626+00,2024-01-13 07:52:15.634626+00,NaN,\N,1a3orn,99.0,0.0,\N


In [2]:
links['added'].max()

'2024-01-13 07:52:15.635604+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
108341    172848
108342    172852
108343    172853
108344    172854
108345    172855
Name: id, Length: 15645, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred


In [5]:
links[links['url'].duplicated()]['id']

Series([], Name: id, dtype: int64)

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/17
...11/17
...17/17


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108381,173012,https://twitter.com/goodside/status/1745845717...,"Riley Goodside on X: ""Prompt engineering: stil...",\N,\N,2024-01-13 13:45:54.954853,2024-01-13 13:45:54.954859,\N,\N,Custom,86.0,0.0,\N
108382,173013,https://utopianscrapbook.substack.com/p/all-my...,All my (vegan) fitness advice - by Andy Masley,\N,\N,2024-01-13 13:45:54.960524,2024-01-13 13:45:54.960529,\N,\N,Custom,6.0,0.0,\N
108383,173014,https://forum.effectivealtruism.org/posts/YDjz...,Matthew_Barnett's Quick takes — EA Forum,\N,\N,2024-01-13 13:45:54.966180,2024-01-13 13:45:54.966186,\N,\N,Custom,71.0,0.0,\N
108384,173015,https://chat.openai.com/g/g-RR3RCyK8N-ms-smith...,ChatGPT - Ms. Smith - Private Language Teacher,\N,\N,2024-01-13 13:45:54.971927,2024-01-13 13:45:54.971933,\N,\N,Custom,79.0,0.0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108381,173012,https://twitter.com/goodside/status/1745845717...,"Riley Goodside on X: ""Prompt engineering: stil...",\N,\N,2024-01-13 13:45:54.954853,2024-01-13 13:45:54.954859,\N,\N,Custom,86.0,0.0,\N
108382,173013,https://utopianscrapbook.substack.com/p/all-my...,All my (vegan) fitness advice - by Andy Masley,\N,\N,2024-01-13 13:45:54.960524,2024-01-13 13:45:54.960529,\N,\N,Custom,6.0,0.0,\N
108383,173014,https://forum.effectivealtruism.org/posts/YDjz...,Matthew_Barnett's Quick takes — EA Forum,\N,\N,2024-01-13 13:45:54.966180,2024-01-13 13:45:54.966186,\N,\N,Custom,71.0,0.0,\N
108384,173015,https://chat.openai.com/g/g-RR3RCyK8N-ms-smith...,ChatGPT - Ms. Smith - Private Language Teacher,\N,\N,2024-01-13 13:45:54.971927,2024-01-13 13:45:54.971933,\N,\N,Custom,79.0,0.0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108381,173012,https://twitter.com/goodside/status/1745845717...,"Riley Goodside on X: ""Prompt engineering: stil...",\N,\N,2024-01-13 13:45:54.954853,2024-01-13 13:45:54.954859,\N,\N,Custom,86,0,\N
108382,173013,https://utopianscrapbook.substack.com/p/all-my...,All my (vegan) fitness advice - by Andy Masley,\N,\N,2024-01-13 13:45:54.960524,2024-01-13 13:45:54.960529,\N,\N,Custom,6,0,\N
108383,173014,https://forum.effectivealtruism.org/posts/YDjz...,Matthew_Barnett's Quick takes — EA Forum,\N,\N,2024-01-13 13:45:54.966180,2024-01-13 13:45:54.966186,\N,\N,Custom,71,0,\N
108384,173015,https://chat.openai.com/g/g-RR3RCyK8N-ms-smith...,ChatGPT - Ms. Smith - Private Language Teacher,\N,\N,2024-01-13 13:45:54.971927,2024-01-13 13:45:54.971933,\N,\N,Custom,79,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)